# Deep Learning for Flower Image Classification

In [5]:
import numpy as np
from keras.applications import vgg16
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers

Deep networks have a large number of unknown parameters (in millions). The task of training a network is to find the optimum parameters using the training data. From linear algebra, we know that in order to solve an equation with three unknown parameters, we need three equations (data). Similarly, for finding all the unknown parameters accurately, we would need a lot of data (in millions). If we have very few data, we will get only approximate values for most of the parameters, which we don’t want.

However, it is difficult to get such huge labeled datasets for training the network, and, even if you get the data, it takes a large amount of time to train the network (hundreds of hours). Fortunately, we can leverage the models already trained on very large amounts of data for difficult tasks with thousands of classes. Most often we use these models as a starting point for our training process, instead of training our own model from scratch.

This notebook uses the pre-trained model *VGG16* (Simonyan K, Zisserman A, 2015) with the initial weights as the ones obtained in the ImageNet ILSVRC-2014 competition. This code is based on the tutorial by Satya Mallick in www.learnopencv.com

## 1. Loading pre-trained model

Load the VGG Model along with the ImageNet weights. The *include_top=False* argument means it does not load the last two fully connected layers which act as the classifier, only the convolutional layers (feature extraction).
The images we are going to load are all 256 x 256 pixels, with RGB (3 channels)

In [6]:
vgg_conv = vgg16.VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(256,256,3))

## 2. Loading images

We will use the *ImageDataGenerator* class from Keras to load the images and *flow_from_directory* function to generate batches of images and labels. The function returns a *DirectoryIterator* yielding tuples of (x, y) where x is a numpy array containing a batch of images with shape (batch_size, target_size, channels) and y is a numpy array of corresponding labels.

In [7]:
train_dir = '/media/thabata/ExtraDrive1/INFNET/Flower_spotter/subset_train'
validation_dir = '/media/thabata/ExtraDrive1/INFNET/Flower_spotter/subset_test'
 
nTrain = 723
nVal = 239

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [8]:
#load images and generate batch for train data

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

Found 723 images belonging to 3 classes.


In [9]:
#load images and generate batch for validation data
 
val_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

Found 239 images belonging to 3 classes.


## 3. Passing images through feature extraction layers

Use *model.predict()* function to pass the images through the pre-trained network feature extraction layers. The 256x256x3 input becomes a 8x8x512 tensor in the final layer.
Finally, we reshape the Tensor into a vector.

In [10]:
# for train data

train_features = np.zeros(shape=(nTrain, 8, 8, 512))
train_labels = np.zeros(shape=(nTrain,3))

i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break
         
train_features = np.reshape(train_features, (nTrain, 8 * 8 * 512))

In [11]:
# for validation data

val_features = np.zeros(shape=(nVal, 8, 8, 512))
val_labels = np.zeros(shape=(nVal,3))

i = 0
for inputs_batch, labels_batch in val_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    val_features[i * batch_size : (i + 1) * batch_size] = features_batch
    val_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nVal:
        break
         
val_features = np.reshape(val_features, (nVal, 8 * 8 * 512))

## 4. Creating classification layers

Create a simple feedforward network with a:

###### a. Relu activation function
###### b. Dropout layer to deactivate half the neurons (minimize overfitting)
###### c. Softmax output layer having 3 classes

In [12]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=8 * 8 * 512))
model.add(layers.Dropout(0.5)) # minimize overfitting (deactivates half of the neurons)
model.add(layers.Dense(3, activation='softmax'))

## 5. Compiling model with optimizer

Configure the model for training, define optimizer to tune the hyperparameters, and defines performance metrics to be applied both in train and validation data.

In [13]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

## 6. Training model

Train the model compiled with the train data and defines epochs, and calculate performance metrics

In [14]:
history = model.fit(train_features,
                    train_labels,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(val_features,val_labels))

Train on 723 samples, validate on 239 samples
Epoch 1/20
723/723 [==============================] - 2s 3ms/step - loss: 0.9212 - acc: 0.7621 - val_loss: 0.1503 - val_acc: 0.9498
Epoch 2/20
723/723 [==============================] - 2s 3ms/step - loss: 0.2458 - acc: 0.9142 - val_loss: 0.0996 - val_acc: 0.9665
Epoch 3/20
723/723 [==============================] - 2s 3ms/step - loss: 0.1225 - acc: 0.9613 - val_loss: 0.0940 - val_acc: 0.9707
Epoch 4/20
723/723 [==============================] - 2s 3ms/step - loss: 0.0814 - acc: 0.9723 - val_loss: 0.1900 - val_acc: 0.9372
Epoch 5/20
723/723 [==============================] - 2s 3ms/step - loss: 0.0684 - acc: 0.9737 - val_loss: 0.1164 - val_acc: 0.9665
Epoch 6/20
723/723 [==============================] - 2s 3ms/step - loss: 0.0678 - acc: 0.9723 - val_loss: 0.0594 - val_acc: 0.9791
Epoch 7/20
723/723 [==============================] - 2s 3ms/step - loss: 0.0228 - acc: 0.9903 - val_loss: 0.1195 - val_acc: 0.9665
Epoch 8/20
723/723 [==========

## 7. Check performance

In [19]:
fnames = val_generator.filenames
ground_truth = val_generator.classes
label2index = val_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
predictions = model.predict_classes(val_features)
prob = model.predict(val_features)
 
errors = np.where(predictions != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),nVal))

No of errors = 157/239
